## Grids

Es gibt viele Problemstellungen, bei denen wir durch eine 2D-Spielfeld (grid) navigieren müssen.

Beispiel: Wir wollen den kürzesten Pfad vom Start zum Ziel finden.

    #:  Wand
    S:  Start
    G:  Ziel (goal)


In [29]:
%%writefile maze1.txt
##########
#   #    #
#   ##   #
# #Z#  ###
# ## #   #
#   S #  #
#     #  #
#        #
##########

Overwriting maze1.txt


#### Einlesen des Grids

Wenn wir das grid nicht verändern wollen, können wir die grid-Daten als eine Liste von Strings einlesen.

In [30]:
f = open('maze1.txt')       
grid = f.read().splitlines()
f.close()
grid

['##########',
 '#   #    #',
 '#   ##   #',
 '# #Z#  ###',
 '# ## #   #',
 '#   S #  #',
 '#     #  #',
 '#        #',
 '##########']

Wir ermitteln die Höhe und Breite des Grids.

In [31]:
height = len(grid)
width = len(grid[0])
print(f'Höhe = {height}, Breite = {width}')

Höhe = 9, Breite = 10


#### Koordinaten im Grid

Der Startpunkt ist in der Zeile mit Index 5, Spalte mit Index 4

In [32]:
grid[5][4]             # Zeile mit Index 5, Spalte mit Index 4

'S'

Die Koordinaten können je nach Problemstellung unterschiedliche Positionen bedeuten, je nachdem wo der Nullpunkt liegt und in welche Richtung die Achsen zeigen. 

Häufig ist folgendes Setting: Der Nullpunkt liegt oben links und wird mit (0,0) bezeichnet. Die x-Achse zeigt nach rechts, die y-Achse nach unten. Damit hat der Punkt S die Koordinaten (4,5). 
Für dieses Setting gilt dann: An der Koordinate (x,y) steht der Wert grid[y][x].

Zur Vereinfachung der folgenden Überlegungen vereinbaren wir:
Mit *Zeile x* bezeichnen wir die Zeile mit Index x. Mit *Spalte y* bezeichnen wir die Spalte mit Index y. Die Koordinate (x,y) oder Position (x,y) bezeichnet die Stelle in Zeile x und Spalte y.
Damit zeigt also die x-Achse nach unten und die y-Achse nach rechts. An der Koordinate (x,y) steht der Wert grid[x][y].


#### Start und Ziel ermitteln    

In [33]:
for x in range(height):
    for y in range(width):
        if grid[x][y] == 'S':
            start = (x,y)
        elif grid[x][y] == 'Z':
            ziel = (x,y)

print(f'{start=}, {ziel=}')

start=(5, 4), ziel=(3, 3)


#### Nachbarschaft

In unseren Algorithmen benötigen wir immer wieder die Nachbarn zu einer bestimmten Position (=Koordinate).
Wir schreiben uns dazu eine Funktion *nb*. Wir gehen im folgenden davon aus, dass das grid sich nicht verändert. Deshalb nutzen wir (zur Vereinfachung) *grid*, *height* und *width* als globale Variablen. Außerdem spendieren wir uns noch eine globale Variable *dirs* mit den möglichen Bewegungsrichtungen


In [34]:
dirs = [(0,-1),(0,1),(-1,0),(1,0)]     # NSWE = hoch,runter,links,rechts

def nb(pos):
    '''
    pos: Tuple (x,y) - die Position im grid
    returns: Liste mit den möglichen Nachbarpositionen
    '''
    x, y = pos
    tmp = []
    for xd, yd in dirs:
        x1 = x + xd
        y1 = y + yd
        if 0 <= x1 < height and 0 <= y1 < width and grid[x1][y1] != '#':
            tmp.append((x1,y1))
    return tmp    

In [35]:
# Beispiele
print(nb((4,1)))
print(nb((5,1)))

[(3, 1), (5, 1)]
[(5, 2), (4, 1), (6, 1)]


### BFS Breitensuche

Die Breitensuche findet einen kürzesten Weg vom startstate zu einem goalstate. Das allgemeine Muster für die Breitensuche:

In [36]:
from collections import deque
def bfs(startstate):
    ''' 
    returns: Tuple (prev, state) 
        prev: dictionary mit den Vorgängern der untersuchten Spielstellungen,            
        state: Spielstellung, die den goaltest besteht
        wenn Ziel nicht gefunden: None, None
    '''   
    frontier =  deque([startstate])
    prev = {startstate:None}
    while frontier:
        state = frontier.popleft()  
        if goaltest(state):
            return prev, state
        for v in nextstates(state):
            if v not in prev:
                frontier.append(v)
                prev[v] = state
    return None, None

def reconstructPath(prev,goalstate):
    state = goalstate
    path = []
    while state is not None:
        path.append(state)
        state = prev[state]
    path.reverse()
    return path

def nextstates(state):
    '''
    state: Spielstellung
    returns:  Liste mit möglichen Folgestellungen zu state
    '''

def goaltest(state):
    '''
    state: Spielstellung
    returns: True, wenn state eine Lösung ist
    '''
    
#Aufruf:
#start = ...
#prev, goal = bfs(start)
#path = reconstructPath(prev,goal) 

#### Breitensuche für das Grid

Im konkreten Fall sehen die zu implementieren Funktionen wie folgt aus:

In [37]:
def nextstates(state):
    return nb(state)

def goaltest(state):
    return state == ziel


prev, goal = bfs(start)
path = reconstructPath(prev,goal)
print(path)

[(5, 4), (5, 3), (5, 2), (5, 1), (4, 1), (3, 1), (2, 1), (2, 2), (2, 3), (3, 3)]


Häufig möchte man die Folge von Aktionen nennen, die diesem Weg entspricht. Dazu implementieren wir noch die Funktion *getMove*

In [38]:
def getMove(s1, s2):
    '''
    returns: die Beschreibung des Übergangs von state s1 zu state s2
    '''
    x1,y1 = s1
    x2,y2 = s2
    if x1 < x2: return 'D'   # down
    if x1 > x2: return 'U'   # up
    if y1 < y2: return 'R'   # right
    if y1 > y2: return 'L'   # left

def getMoves(path):
    '''
    returns: Beschreibung des Pfads als eine Folge von Aktionen (Moves)
    '''
    moves = ''
    for i in range(len(path)-1):
        moves+=getMove(path[i],path[i+1])
    return moves

In [39]:
prev, goal = bfs(start)
path = reconstructPath(prev,goal)
print(getMoves(path))

LLLUUURRD


Wir lassen uns den gefunden Weg im Grid anzeigen. Die untersuchten Koordinaten in prev werden mit '.' bezeichnet.

In [40]:
def showGrid(path, prev):
    grid0 = [list(s) for s in grid]
    for x,y in prev:
        grid0[x][y]='.'
    for x,y in path:
        grid0[x][y] = 'o'
    x,y = path[0]
    grid0[x][y] = 'S'
    x,y = path[-1]
    grid0[x][y] = 'Z'
    for row in grid0:
        print(''.join(row))
 
    print(f'prev = {len(prev)}, weglänge = {len(path)-1}')
    print("gefundener Weg: 'o', prev: '.'")

showGrid(path,prev)

##########
#...#    #
#ooo##   #
#o#Z#  ###
#o##.#...#
#oooS.#..#
#.....#..#
#........#
##########
prev = 35, weglänge = 9
gefundener Weg: 'o', prev: '.'


#### Breitensuche fürs Grid - kompakte Version

In [64]:
from collections import deque
def bfsPath(grid,startChar='S',goalChars='Z',wallChar='#'):
    def bfs(startstate):
        frontier =  deque([startstate])
        prev = {startstate:None}
        while frontier:
            x, y = frontier.popleft()  
            if grid[x][y] in goalChars:
                return prev, (x,y)
            for xd, yd in dirs:
                x1 = x + xd
                y1 = y + yd
                if 0 <= x1 < height and 0 <= y1 < width and grid[x1][y1] != wallChar:
                    if (x1,y1) not in prev:
                        frontier.append((x1,y1))
                        prev[(x1,y1)] = (x,y)
        return None, None
    
    def reconstructPath(prev,goalstate):
        state = goalstate
        path = []
        while state is not None:
            path.append(state)
            state = prev[state]
        path.reverse()
        return path

    dirs = [(0,-1),(0,1),(-1,0),(1,0)]
    height = len(grid)
    width = len(grid[0])
    for x in range(height):
        for y in range(width):
            if grid[x][y] == startChar:
                start = (x,y)
 
    prev, goal = bfs(start)
    if not prev: return []                    # kein Pfad zum Ziel gefunden
    return reconstructPath(prev,goal)

bfsPath(grid) 

[(5, 4),
 (5, 3),
 (5, 2),
 (5, 1),
 (4, 1),
 (3, 1),
 (2, 1),
 (2, 2),
 (2, 3),
 (3, 3)]

(5, 4)


[]

In [61]:
grid[3][3] in 'Z'

True

In [41]:
def bfsLevels(startstate):
    frontier =  [startstate]
    next_frontier = []
    prev = {startstate:None}
    zaehl = 0
    while frontier:
        while frontier:
            state = frontier.pop()  
            grid0[state[0]][state[1]] = zaehl
            if goaltest(state):
                return grid0
            for v in nextstates(state):
                if v not in prev:
                    next_frontier.append(v)
                    
                    prev[v] = state
        frontier = next_frontier
        next_frontier = []
        zaehl = (zaehl+1)%10
    return None

In [42]:
showGrid(path,prev)

##########
#...#    #
#ooo##   #
#o#Z#  ###
#o##.#...#
#oooS.#..#
#.....#..#
#........#
##########
prev = 35, weglänge = 9
gefundener Weg: 'o', prev: '.'


In [ ]:
def bfsPath(grid,startchar='S',goalChar='Z',wallChar='#'):
    

In [75]:
def showBfs(grid,startChar='S',goalChars='Z',wallChar='#'):

    def bfs(startstate):
        
        frontier = [startstate]
        next_frontier = []
        prev = {startstate:None}
        level = 0
        while frontier:
            while frontier:
                pos = frontier.pop()  
                x, y = pos
                if grid1[x][y] in goalChars:
                    print(f'frontier level = {level} = {chr(ord("a")+level)}')
                    for (x1,y1) in frontier:                        
                        grid1[x1][y1] = '~'
                    return pos, prev, grid1, frontier
                if (x,y) != startstate:
                    grid1[x][y] = chr(ord('a')+level)
                for xd, yd in dirs:
                    x1 = x + xd
                    y1 = y + yd
                    if 0 <= x1 < height and 0 <= y1 < width and grid1[x1][y1] != wallChar and (x1,y1) not in prev:
                        next_frontier.append((x1,y1))
                        prev[(x1,y1)] = pos
            frontier = next_frontier
            next_frontier = []
            level = (level+1)%26
        return None, None, None, None

    def reconstructPath(prev,goalstate):
        state = goalstate
        path = []
        while state is not None:
            path.append(state)
            state = prev[state]
        path.reverse()
        return path

    grid1 = [list(s) for s in grid]
    dirs =  [(0,-1),(0,1),(-1,0),(1,0)]

    # Höhe und Breite und Startposition ermitteln
    height = len(grid)
    width = len(grid[0])
    for x in range(height):
        for y in range(width):
            if grid1[x][y] == startChar:
                start = (x,y)

    goal, prev, grid1, frontier = bfs(start)
    path = reconstructPath(prev,goal)
 
    for zeile in grid1:
        print(*zeile,sep='')

    grid2 = [list(s) for s in grid]
    for x in range(height):
        for y in range(width):
            if (x,y) in prev:
                grid2[x][y] = '.'
            if (x,y) in frontier:
                grid2[x][y] = '~'
            if (x,y) in path:
                grid2[x][y] = 'o'
            x1,y1 = start
            grid2[x1][y1] = 'S'
            x2,y2 = goal
            grid2[x2][y2] = 'Z'

    print()
    for zeile in grid2:
        print(*zeile,sep='')



In [78]:

f = open('maze3.txt')       
grid = f.read().splitlines()
f.close()
showBfs(grid)

frontier level = 9 = j
###################################################################
#Sbcdefghijklmnopqrstuvwxyzabcdefghijklmnopqrstuvwxyzabcdefghijklm#
#bcdefghijklmnopqrstuvwxyzabcdefghijklmnopqrstuvwxyzabcdefghijklmn#
#cdefghijklmnopqrstuvwxyzabcdefghijklmnopqrstuvwxyzabcdefghijklmno#
#defghijklmnopqrstuvwxyzabcdefghijklmnop#rstuvwxyzabcdefghijklmnop#
#efghijklmnopqrstuvwxyzabcdefghijklmnopq#stuvwxyzabcdefghijklmnopq#
#fghijklmnopqrstuvwxyzabcdefghijklmnopqr#tuvwxyzabcdefghijklmnopqr#
#ghijklmnopqrstuvwxyzabcdefghijklmnopqrs#uvwxyzabcdefghijklmnopqrs#
#hijklmnopqrstuvwxyzabcdefghijklmnopqrst#############hijklmnopqrst#
#ijklmnopqrstuvwxyzabcdefghijklmnopqrstu#utsrqponmlkjijklmnopqrstu#
#jklmnopqrstuvwxyzabcdefghijklmnopqrstuv#vutsrqponmlkjklmnopqrstuv#
#klmnopqrstuvwxyzabcdefghijklmnopqrstuvw#wvutsrqponmlklmnopqrstuvw#
#lmnopqrstuvwxyzabcdefghijklmnopqrstuvwx#vwvutsrqponmlmnopqrstuvwx#
#mnopqrstuvwxyzabcdefghijklmnopqrstuvwxy#uvwvutsrqponmnopqrstuvwxy#
#nopqrstuvwxyzabcdefghijk

In [162]:
def getPath(grid,startChar='S',goalChars='Z',wallChar='#'):

    def bfs(startstate):
        frontier = [startstate]
        next_frontier = []
        prev = {startstate:None}
        while frontier:
            while frontier:
                x, y = frontier.pop()  
                if grid[x][y] in goalChars:
                    return prev, (x,y)
                for xd, yd in dirs:
                    x1 = x + xd
                    y1 = y + yd
                    if 0 <= x1 < height and 0 <= y1 < width and grid[x1][y1] != wallChar and (x1,y1) not in prev:
                        next_frontier.append((x1,y1))
                        prev[(x1,y1)] = (x,y)
            frontier = next_frontier
            next_frontier = []
        return None, None 

    def reconstructPath(prev,goalstate):
        state = goalstate
        path = []
        while state is not None:
            path.append(state)
            state = prev[state]
        path.reverse()
        return path

    dirs =  [(0,-1),(0,1),(-1,0),(1,0)]
   
    
    height = len(grid)
    width = len(grid[0])
    for x in range(height):
        for y in range(width):
            if grid[x][y] == startChar:
                start = (x,y)

    prev, goal = bfs(start)
    return reconstructPath(prev,goal)

In [163]:
f = open('maze3.txt')       
grid = f.read().splitlines()
f.close()
getPath(grid)

[(1, 1),
 (1, 2),
 (1, 3),
 (1, 4),
 (1, 5),
 (1, 6),
 (1, 7),
 (1, 8),
 (1, 9),
 (1, 10),
 (1, 11),
 (1, 12),
 (1, 13),
 (1, 14),
 (1, 15),
 (1, 16),
 (1, 17),
 (1, 18),
 (1, 19),
 (1, 20),
 (1, 21),
 (1, 22),
 (1, 23),
 (1, 24),
 (1, 25),
 (1, 26),
 (1, 27),
 (1, 28),
 (1, 29),
 (1, 30),
 (1, 31),
 (1, 32),
 (1, 33),
 (1, 34),
 (1, 35),
 (1, 36),
 (1, 37),
 (1, 38),
 (1, 39),
 (1, 40),
 (1, 41),
 (1, 42),
 (1, 43),
 (1, 44),
 (1, 45),
 (1, 46),
 (1, 47),
 (2, 47),
 (2, 48),
 (3, 48),
 (3, 49),
 (4, 49),
 (4, 50),
 (5, 50),
 (5, 51),
 (6, 51),
 (6, 52),
 (7, 52),
 (7, 53),
 (8, 53),
 (9, 53),
 (10, 53),
 (11, 53),
 (12, 53),
 (13, 53),
 (13, 54),
 (13, 55),
 (14, 55),
 (14, 56),
 (15, 56),
 (15, 57),
 (16, 57),
 (16, 58),
 (17, 58),
 (17, 59),
 (18, 59),
 (18, 60),
 (19, 60),
 (19, 61),
 (20, 61),
 (20, 62),
 (21, 62),
 (21, 63),
 (22, 63),
 (22, 64),
 (23, 64),
 (23, 65),
 (24, 65)]

In [252]:
def bfs(start,ziel):
    frontier = [start]
    next_frontier = []
    prev = {start:None}
    while frontier:
        while frontier:
            x, y = frontier.pop()  
            if (x,y) == ziel:
                return prev 
            for xd, yd in dirs:
                x1 = x + xd
                y1 = y + yd
                if 0 <= x1 < height and 0 <= y1 < width and grid[x1][y1] != '#' and (x1,y1) not in prev:
                    next_frontier.append((x1,y1))
                    prev[(x1,y1)] = (x,y)
        frontier = next_frontier
        next_frontier = []
    return None

def bibfs(start,ziel):
    frontier = [start]
    frontier2 = [ziel]
    next_frontier = []
    next_frontier2 = []
    prev = {start:None}
    prev2 = {ziel:None}
    while frontier or frontier2:
        while frontier:
            x, y = frontier.pop()  

            for xd, yd in dirs:
                x1 = x + xd
                y1 = y + yd
                if 0 <= x1 < height and 0 <= y1 < width and grid[x1][y1] != '#' and (x1,y1) not in prev:
                    next_frontier.append((x1,y1))

                    prev[(x1,y1)] = (x,y)

        frontier = next_frontier
        next_frontier = []

        while frontier2:
            x, y = frontier2.pop()  
            for xd, yd in dirs:
                x1 = x + xd
                y1 = y + yd
                if 0 <= x1 < height and 0 <= y1 < width and grid[x1][y1] != '#' and (x1,y1) not in prev2:
                    next_frontier2.append((x1,y1))
                    prev2[(x1,y1)] = (x,y)
                    if (x1,y1) in prev:
                        return prev, prev2, (x1,y1) 
        frontier2 = next_frontier2
        next_frontier2 = []
    return None

def reconstructPath(prev,goalstate):
    state = goalstate
    path = []
    while state is not None:
        path.append(state)
        state = prev[state]
    path.reverse()
    return path

In [253]:
f = open('maze3.txt')       
grid = f.read().splitlines()
f.close()
for zeile in grid:
    print(zeile)
dirs =  [(0,-1),(0,1),(-1,0),(1,0)]

startChar = 'S'
goalChar = 'Z'

height = len(grid)
width = len(grid[0])
for x in range(height):
    for y in range(width):
        if grid[x][y] == startChar:
            start = (x,y)
        elif grid[x][y] == goalChar:
            ziel = (x,y)
height, width, start, ziel          

###################################################################
#S                                                                #
#                                                                 #
#                                                                 #
#                                       #                         #
#                                       #                         #
#                                       #                         #
#                                       #                         #
#                                       #############             #
#                                       #                         #
#                                       #                         #
#                                       #                         #
#                                       #                         #
#                                       #                         #
#                                       #       

(26, 67, (1, 1), (24, 65))

In [254]:
print(f'{start=}, {ziel=}')
prev, prev2, mid = bibfs(start,ziel)
print(f'{mid=}')
path1 = reconstructPath(prev, mid)
path2 = reconstructPath(prev2,mid)
print(f'prev={len(prev)+len(prev2)}')
pathA = path1 + path2[::-1][1:]
print(pathA)
print(len(pathA))

start=(1, 1), ziel=(24, 65)
mid=(1, 44)
prev=1379
[(1, 1), (1, 2), (1, 3), (1, 4), (1, 5), (1, 6), (1, 7), (1, 8), (1, 9), (1, 10), (1, 11), (1, 12), (1, 13), (1, 14), (1, 15), (1, 16), (1, 17), (1, 18), (1, 19), (1, 20), (1, 21), (1, 22), (1, 23), (1, 24), (1, 25), (1, 26), (1, 27), (1, 28), (1, 29), (1, 30), (1, 31), (1, 32), (1, 33), (1, 34), (1, 35), (1, 36), (1, 37), (1, 38), (1, 39), (1, 40), (1, 41), (1, 42), (1, 43), (1, 44), (1, 45), (2, 45), (2, 46), (3, 46), (3, 47), (4, 47), (4, 48), (5, 48), (5, 49), (6, 49), (6, 50), (7, 50), (7, 51), (7, 52), (7, 53), (8, 53), (8, 54), (9, 54), (9, 55), (10, 55), (10, 56), (11, 56), (11, 57), (12, 57), (12, 58), (13, 58), (13, 59), (14, 59), (14, 60), (15, 60), (15, 61), (16, 61), (16, 62), (17, 62), (17, 63), (18, 63), (18, 64), (18, 65), (19, 65), (20, 65), (21, 65), (22, 65), (23, 65), (24, 65)]
88


In [255]:
print(f'{start=}, {ziel=}')
prev  = bfs(start,ziel)
pathB = reconstructPath(prev, ziel)
print(f'prev={len(prev)}')
print(pathB)
print(len(pathB))

start=(1, 1), ziel=(24, 65)
prev=1467
[(1, 1), (1, 2), (1, 3), (1, 4), (1, 5), (1, 6), (1, 7), (1, 8), (1, 9), (1, 10), (1, 11), (1, 12), (1, 13), (1, 14), (1, 15), (1, 16), (1, 17), (1, 18), (1, 19), (1, 20), (1, 21), (1, 22), (1, 23), (1, 24), (1, 25), (1, 26), (1, 27), (1, 28), (1, 29), (1, 30), (1, 31), (1, 32), (1, 33), (1, 34), (1, 35), (1, 36), (1, 37), (1, 38), (1, 39), (1, 40), (1, 41), (1, 42), (1, 43), (1, 44), (1, 45), (1, 46), (1, 47), (2, 47), (2, 48), (3, 48), (3, 49), (4, 49), (4, 50), (5, 50), (5, 51), (6, 51), (6, 52), (7, 52), (7, 53), (8, 53), (9, 53), (10, 53), (11, 53), (12, 53), (13, 53), (13, 54), (13, 55), (14, 55), (14, 56), (15, 56), (15, 57), (16, 57), (16, 58), (17, 58), (17, 59), (18, 59), (18, 60), (19, 60), (19, 61), (20, 61), (20, 62), (21, 62), (21, 63), (22, 63), (22, 64), (23, 64), (23, 65), (24, 65)]
88


In [249]:
pathA == pathB

True

In [250]:
for i in range(len(path)):
    if pathA[i] != pathB[i]:
        print(i,pathA[i], pathB[i])

In [251]:
showPath(grid,pathA)

###################################################################
#ooooooooooooooooooooooooooooooooooooooooooooooo                  #
#                                              oo                 #
#                                               oo                #
#                                       #        oo               #
#                                       #         oo              #
#                                       #          oo             #
#                                       #           oo            #
#                                       #############o            #
#                                       #            o            #
#                                       #            o            #
#                                       #            o            #
#                                       #            o            #
#                                       #            ooo          #
#                                       #       

In [244]:
def showPath(grid,path):
    grid0 = [list(s) for s in grid] 
    for (x,y) in path:
        grid0[x][y] = 'o'
    for zeile in grid0:
        print(*zeile,sep='')
    

In [245]:
showPath(grid,pathB)

###################################################################
#                                                                 #
#                                                                 #
#                                                                 #
#                                       #                         #
#                                       #                         #
#                                       #                         #
#                                       #                         #
#                                       #############             #
#                                       #                         #
#                                       #                         #
#                                       #                         #
#                                       #                         #
#                                       #                         #
#                                       #       